In [ ]:
# --- Install Package ---
!pip install pandas rapidfuzz openpyxl wordninja

In [7]:
import pandas as pd
import re
from rapidfuzz import fuzz, process
import wordninja

# --- CONFIG ---
excel_file = "fuzzy_match_template.xlsx"
output_file = "matched_result.xlsx"

# --- LOAD DATA ---
df1 = pd.read_excel(excel_file, sheet_name="Names_To_Match")
df2 = pd.read_excel(excel_file, sheet_name="Reference_Names")

# --- CLEANING FUNCTION ---
def clean_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower()

    # Ambil isi dalam kurung (jika ada)
    parens = re.findall(r"\((.*?)\)", name)
    paren_name = parens[0] if parens else ""

    # Hapus isi dalam tanda kurung dari nama asli
    name = re.sub(r"\(.*?\)", "", name)

    # Ganti 'm' atau 'm.' dengan 'muhammad'
    name = re.sub(r"\bm[\.\s]", "muhammad ", name)

    # Ganti simbol, tanda baca jadi spasi
    name = re.sub(r"[^\w\s]", " ", name)
    name = re.sub(r"\s+", " ", name).strip()

    # ⬅️ Add word segmentation for strings with no spaces
    if " " not in name:
        name = " ".join(wordninja.split(name))

    # Tambahkan nama dalam tanda kurung jika belum ada
    if paren_name and paren_name.lower() not in name:
        name = paren_name.lower() + " " + name

    return name

# --- CLEAN DATA ---
df1["clean_name"] = df1["UserName"].apply(clean_name)
df2["clean_lookup"] = df2["Name"].apply(clean_name)

# --- FUZZY MATCHING ---
def fuzzy_match(name, choices, threshold=85):
    match = process.extractOne(name, choices, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= threshold:
        return match[0]
    return None

df2["matched_from_df1"] = df2["clean_lookup"].apply(lambda x: fuzzy_match(x, df1["clean_name"].tolist()))
df2 = df2.merge(df1[["UserName", "clean_name"]], left_on="matched_from_df1", right_on="clean_name", how="left")

# --- EXPORT: Only show essential columns ---
final_output = df2[["UserName", "Name"]]
final_output.to_excel(output_file, index=False)

print(f"✅ Matching completed! Output saved to {output_file}")


✅ Matching completed! Output saved to matched_result.xlsx
